In [229]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [230]:
df = pd.read_csv("Data_120294_2023-03-29.csv")

/var/folders/k_/917q0lsn2g199rm9y4s8jy5h0000gn/T/ipykernel_68218/922264029.py:1: DtypeWarning: Columns (2,64,91,94,97,106,108,109,110,111,112,114,116,119,120,122,127,139,142,143,144,145,152,153,154,155,156,157,159,160,161,163,164,165,169,170,171,172,175,178,179,180,187,189,215,216,217,218,219,317,318) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data_120294_2023-03-29.csv")


In [231]:
from missingpy import MissForest

pre_drop = len(df.columns)
df = df.select_dtypes(exclude=['object'])   # Remove columns that do not contain numeric values
df = df.dropna(axis=1, how='all')           # Drop columns that has 100% missing values
print(f"Dropped {pre_drop - len(df.columns)}/{pre_drop} ({round((pre_drop - len(df.columns)) / pre_drop * 100, 2)}%) -> {len(df.columns)} columns left")


Dropped 79/320 (24.69%) -> 241 columns left


In [232]:
df = df[['age', 'bmi_tcr', 'wgt_kg_calc', 'hgt_cm_calc']]
#df = df.dropna(subset=['age'])

# create a new df where x is the column name and y is the number of missing values
missing_values = pd.DataFrame(df.isnull().sum(), columns=['missing_values'])
missing_values['missing_values_percent'] = round(missing_values['missing_values'] / len(df) * 100, 2)
missing_values = missing_values.sort_values(by='missing_values', ascending=False)
missing_values



,missing_values,missing_values_percent
hgt_cm_calc,43621,36.27
wgt_kg_calc,43422,36.11
age,42861,35.64
bmi_tcr,4587,3.81


In [233]:
df = df.dropna(subset=['wgt_kg_calc', 'age', 'hgt_cm_calc'])
cols_with_missing = [col for col in df.columns if df[col].isnull().any()] # Get columns with missing values
print(f"Imputing {len(cols_with_missing)} columns with {df.isnull().sum().sum()} missing values")

rows_bmi_tcr_missing = df[df['bmi_tcr'].isnull()].index

y = MissForest().fit_transform(df) # Impute
df = pd.DataFrame(y, columns=df.columns)
print(f"Imputed {len(cols_with_missing)} columns, now with {df.isnull().sum().sum()} missing values")

# drop all rows except the ones that had missing values in the bmi_tcr column
df = df.drop(df.index.difference(rows_bmi_tcr_missing))
# compute column "true_bmi"
df['true_bmi'] = df['wgt_kg_calc'] / (df['hgt_cm_calc'] / 100) ** 2
# compute column "bmi_diff"
df['bmi_diff'] = df['bmi_tcr'] - df['true_bmi']
# compute column "bmi_diff_percent"
df['bmi_diff_percent'] = df['bmi_diff'] / df['true_bmi'] * 100
df


Imputing 1 columns with 769 missing values
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Imputed 1 columns, now with 0 missing values


ValueError: Cannot take a larger sample than population when 'replace=False'